In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import patsy
from sklearn.preprocessing import StandardScaler
import warnings
warnings.simplefilter('ignore')


### 1 Loading the data

In [11]:
os.chdir("//your_directory")

data = pd.read_csv('wage2015_subsample_inference.csv')

X = data.drop(['wage', 'lwage'], axis=1)
y = data[['lwage']]

In [12]:
X_flexible = patsy.dmatrix('0+ sex + (exp1+exp2+exp3+exp4+hsg+scl+clg+ad+so+we+ne+C(occ2)+C(ind2))**2',
                      X, return_type='dataframe')

In [13]:
y = y.to_numpy()
X_flexible = X_flexible.to_numpy()

### 2 Creating the Lasso Cross-Validation Procedure

4. The `log_grid` function is pretty straight forward

In [14]:
def log_grid(lower:int, upper:int, log_step:int):
    
    log_grid = np.linspace(lower,upper,int(1/log_step))
    
    return np.exp(log_grid)

5. To code the `k_folds` function, there are many different approaches. However, we sticked to using only numpy. With this library, we exploited the kronecker product operation and block matrices to build the $k$-folds. Also, we addressed the issue of divisibility between the sample size $n$ and $k$ using an if-else statement depending on the module of $n/k$

In [15]:
def k_folds(data:np.ndarray, k:int = 5):

    
    module = data.shape[0]%k
    floor = data.shape[0]//k 

    if module == 0:
        
        trues = np.repeat(1, floor).reshape(-1,1)

        split_matrix = np.kron(np.eye(k), trues)
        
    else:
        
        trues_g1 = np.repeat(1,floor+1).reshape(-1,1)

        split_matrix_g1 = np.kron(np.eye(module), trues_g1)

        trues_g2 = np.repeat(1,floor).reshape(-1,1)

        split_matrix_g2 = np.kron(np.eye(k-module), trues_g2)
        
        split_matrix = np.block([[split_matrix_g1, np.zeros((split_matrix_g1.shape[0],split_matrix_g2.shape[1]))],
                                     [np.zeros((split_matrix_g2.shape[0],split_matrix_g1.shape[1])), split_matrix_g2]])
        
    sm_bool = split_matrix == 1
    
    splits = [sm_bool[:,x] for x in range(k)]
        
    return splits

6. For the `optimal_lambda` search function, we basically adapted the code provided in the labs so it can use the functions of log-grid and our own $k$-folds function

In [16]:

def optimal_lambda(Y:np.ndarray, X:np.ndarray, lambda_bounds:tuple, k:int=5, *, niter:int = 100):
    
    from sklearn.linear_model import Lasso
    
    Y = Y.squeeze()
    
    if len(X.shape) == 1:
        
        X = X.reshape(-1,1)
        
    folds = k_folds(X,k)
    
    all_lambdas = log_grid(lambda_bounds[0],lambda_bounds[1],1/niter)
        
    all_mse = np.zeros(niter)

    for l in all_lambdas:
        
        split_pes = np.zeros(k)
        
        for i in range(k):
            
            X_train, X_test = X[~folds[i]],  X[folds[i]]
            y_train, y_test = Y[~folds[i]],  Y[folds[i]]

            
            model = Lasso(alpha=l).fit(X_train, y_train)
            
            predict = model.predict(X_test)
            
            pe = np.sum((y_test-predict)**2)
            
            split_pes[i] = pe
            
        all_mse[all_lambdas == l] = np.mean(split_pes)

    selected = np.where(all_mse == np.min(all_mse))

    optimal_lambda = all_lambdas[selected][0]

    optimal_model = Lasso(alpha=optimal_lambda).fit(X,Y)
    
    optimal_coef = np.hstack((optimal_model.intercept_,optimal_model.coef_))
    
    output = {'optimal_lambda':optimal_lambda, 'optimal_coef':optimal_coef,
              'all_lambdas': all_lambdas, 'all_mse':all_mse}
    
    return output

7. The `predict_model` function can be easily implemented using the results of `optimal_function`

In [17]:
def predict_model(optimal_model:dict, X:np.ndarray):

    intercept = np.ones((X.shape[0],1))
    Z = np.hstack((intercept,X))
    return Z@optimal_model['optimal_coef'].reshape(-1,1)

### 3 Applying the Lasso Cross-Validation Procedure

We split the sample in train and test

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
X_flexible_train, X_flexible_test, y_train, y_test = train_test_split(X_flexible, y, test_size= 0.2)

8. We perform the OLS fitting

In [20]:
from sklearn.linear_model import LinearRegression
model_ls = LinearRegression()
model_ls.fit(X_flexible_train,y_train)

LinearRegression()

9. Npw we search the optimal lambda using our `optimal_lambda` function

In [21]:
model_lasso = optimal_lambda(y_train,X_flexible_train, (-7,7))

In [22]:
print(model_lasso['all_mse'])

[188.40950564 188.47007834 188.55786338 188.69934753 188.74890247
 188.9163444  189.1831836  189.48311351 190.00958892 190.61145265
 191.49515625 192.48210184 193.33321817 194.34308985 195.38394887
 196.6451077  198.03200694 199.60955166 201.04740548 202.25731594
 203.14112811 203.65789293 204.43406364 204.78745079 205.46874299
 206.26277722 207.12858053 208.21755093 209.57551888 211.3388645
 213.47260199 215.36635659 217.17890811 218.94259382 220.92372155
 223.35292759 225.91778551 228.57790985 231.46705954 234.75894208
 237.53611693 239.72290982 241.57174203 243.84894214 247.54720864
 252.59461926 256.02899806 259.7128986  261.88388117 262.52379559
 263.19961991 263.71888695 264.07612931 264.22124946 264.26437035
 264.2867458  264.31052028 264.34125046 264.38118528 264.43329057
 264.50136379 264.59031838 264.70688046 264.85982171 265.06072435
 265.3250228  265.63090391 265.83037419 266.06464927 266.1944086
 266.22184259 266.23599806 266.24535096 266.24368577 266.23812715
 266.2299760

In [23]:
print(model_lasso['optimal_lambda'])

0.0009118819655545162


In [24]:
print(model_lasso['optimal_coef'])

[ 2.72196751e+00  2.27559832e-01  1.79687331e-01  2.18574838e-01
  2.57395489e-02  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -6.49899878e-02 -0.00000000e+00  8.04129969e-02 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -1.19916477e-01 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -7.44734693e-02  0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -1.28256846e-02 -0.00000000e+00 -0.00000000e+00
  9.84402186e-02  0.00000000e+00  7.70408939e-03 -0.00000000e+00
 -0.00000000e+00 -1.04635824e-01 -1.62160251e-02 -1.05642074e-01
 -2.81035036e-01 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00  0.00000

For comparison, we can see that our code replicates exactly (but less efficiently) the results of the routine embedded in LassoCV

In [25]:
from sklearn.linear_model import LassoCV
model = LassoCV(alphas = log_grid(-7,7,1/100))
fitted = model.fit(X_flexible_train,y_train)

In [26]:
fitted.alpha_

np.float64(0.0010503982945612605)

In [27]:
np.hstack((fitted.intercept_,fitted.coef_))

array([ 2.72043631e+00,  2.23564710e-01,  1.70802463e-01,  2.04884936e-01,
        1.98309629e-02,  0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -7.04791420e-02, -0.00000000e+00,  6.79265097e-02, -0.00000000e+00,
       -0.00000000e+00, -0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        0.00000000e+00, -1.18218745e-01, -0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -5.83410283e-02,  0.00000000e+00,
        0.00000000e+00, -0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -6.54060409e-03, -0.00000000e+00, -0.00000000e+00,
        1.01839414e-01,  0.00000000e+00,  1.58519821e-02, -0.00000000e+00,
       -0.00000000e+00, -8.73004634e-02, -3.77517983e-03, -7.54384105e-02,
       -2.62362798e-01, -0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -

10. Now we use HDM for python (hdmpy) to estimate the model using the theoretically optimal penalty parameter.

In [28]:
# !git clone https://github.com/maxhuppertz/hdmpy
import hdmpy

In [29]:
model_rlasso = hdmpy.rlasso(X_flexible_train,y_train)

As you may notice, the optimal penalty parameter resulting from this procedure is not comparable in size to the cross validation result. This is due to the fact that this penalty is the theoretically optimal parameter for the Lasso estimator under data-driven penalty loadings. That is:

\begin{equation*}

\hat{\beta} = \arg \ \underset{\beta}{\min} \sum_{i=1}^n (y_i - x_{i}^{\prime}\beta)^2 + \frac{\lambda}{n} \lVert \hat{\Psi}\beta \rVert_1

\end{equation*}

Where $\hat{\Psi} = diag(\hat{\psi_1},\hat{\psi_2},\dots,\hat{\psi_p})$ are the data-driven penalty loadings chosen to be a function of the data depending on the setting. For more detail, you can check the [package documentation](https://arxiv.org/pdf/1608.00354)

In [30]:
rlambda = model_rlasso.est['lambda0']
rlambda

np.float64(617.5092395386587)

11. The predictive capability of each model (OLS, Lasso and RLasso) is reported via $MSE$ and $R^2$ out of sample

In [31]:
# OLS 

y_predict_ols = model_ls.predict(X_flexible_test)
MSE_ols = np.mean((y_test-y_predict_ols)**2)
R2_test_ols = 1-MSE_ols/np.var(y_test)
print(R2_test_ols)

0.14661999437037587


In [32]:
# Lasso CV

y_predict_lasso = predict_model(model_lasso, X_flexible_test)
MSE_lasso = np.mean((y_test-y_predict_lasso)**2)
R2_test_lasso = 1-MSE_lasso/np.var(y_test)
print(R2_test_lasso)

0.2412077021708694


In [33]:
# Rigurous Lasso
intercept = np.ones((X_flexible_test.shape[0],1))
Z = np.hstack((intercept,X_flexible_test))
y_predict_rlasso = Z@model_rlasso.est['coefficients'].to_numpy()

MSE_rlasso = np.mean((y_test-y_predict_rlasso)**2)
R2_test_rlasso = 1-MSE_rlasso/np.var(y_test)
print(R2_test_rlasso)

0.25665518392521847
